

优化模型参数
===========================

现在我们有了模型和数据，是时候通过优化模型参数来训练、验证和测试我们的模型了。训练模型是一个迭代过程；在每次迭代中，模型都会对输出进行猜测，计算其猜测的误差（损失），收集误差相对于其参数的导数（正如我们在上一节中看到的），并使用梯度下降优化这些参数。有关此过程的更详细演练，请查看有关来自 3Blue1Brown 的反向传播的视频。[backpropagation from
3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

先决条件代码
-----------------

我们从先前章节中加载代码，这些章节介绍了数据集和数据加载器以及构建模型。


In [131]:
# 导入模块
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [132]:
# 生成数据集
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [133]:
# 封装成数据迭代器
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [134]:
# # 定义网络结构
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(28*28, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, 10),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits
# model = NeuralNetwork()

# 卷积神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(CNN, self).__init__()
        
        # 卷积，输入1/3通道，输出8通道
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)

        # 批量归一化，减少梯度消失或梯度爆炸问题。通过稳定输入分布，模型可以更快收敛，具有一定的正则化作用，减少过拟合风险
        self.bn1 = nn.BatchNorm2d(8)  # 加入 BatchNorm

        # 池化降采样
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 卷积，输入8通道，输出16通道
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)

        # 批量归一化，减少梯度消失或梯度爆炸问题。通过稳定输入分布，模型可以更快收敛，具有一定的正则化作用，减少过拟合风险
        self.bn2 = nn.BatchNorm2d(16)

        # 全局池化，动态调整尺寸
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        # 全连接层
        self.fc1 = nn.Linear(16*28*28, num_classes)

    def forward(self, x):


        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        # x = self.pool(x)


        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        # x = self.pool(x)
      
        # x = self.global_pool(x)  # 全局池化
        print(x.shape)
        x = x.view(x.size(0), -1)  # 展平
        print(x.shape)
        x = self.fc1(x)  # 全连接层
        return x

上述网络结构设计结论：
池化层会导致特征丢失有效信息，影响准确率：

3轮池化下：准确率48.6% 平均损失：1.677764
2轮池化下：准确率86.0% 平均损失：0.400302
1轮池化下：准确率87.1% 平均损失：0.372437
0池化下：准确率88.8% 平均损失：0.311007
去二轮卷积下：准确率87.3% 平均损失：0.371074

由此可见，池化层较严重影响特征信息

In [135]:
model = CNN(in_channels=1, num_classes=10)

超参数
===============

超参数是可以调整的参数，它们可以让你控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（了解更多关于超参数调优的信息）。

我们为训练定义以下超参数：

:   -   **Epochs** - 在数据集上迭代的次数。
    -   **Batch Size** - 在更新参数之前，通过网络传播的数据样本数量。
    -   **Learning Rate** - 在每次批次/时期更新模型参数的幅度。较小的值会导致学习速度缓慢，而较大的值可能会导致训练过程中的不可预测行为


In [136]:
# 定义超参数
learning_rate = 1e-3
batch_size = 64
epochs = 5

优化循环
=================

在设置了超参数后，我们就可以使用优化循环来训练和优化模型。优化循环的每次迭代被称为一个 **epoch**.

每个epoch 包含两个主要部分:

:   -   **The Train Loop** - 在训练数据集上迭代，并尝试收敛到最佳参数。.
    -   **The Validation/Test Loop** - 在测试数据集上迭代，检查模型性能是否有所提升.

损失函数
-------------

当给定一些训练数据时，我们的未训练网络可能无法给出正确的结果。**损失函数**衡量获得的结果与目标值的差异程度，我们希望在训练过程中最小化损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将预测结果与真实数据标签值进行比较。

常见的损失函数包括nn.MSELoss（均方误差），适用于回归任务，以及nn.NLLLoss（负对数似然），适用于分类任务。
nn.CrossEntropyLoss将nn.LogSoftmax和nn.NLLLoss结合在一起。

我们将模型的输出 logits 传递给nn.CrossEntropyLoss，它会对 logits 进行归一化并计算预测误差。


In [137]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

优化器
=========

优化是指调整模型参数以减少每次训练步骤中的模型误差的过程。优化算法定义了如何执行此过程（在本例中，我们使用随机梯度下降）。所有优化逻辑都封装在optimizer对象中。这里，我们使用 SGD 优化器；此外，PyTorch 还提供许多不同的优化器，例如 ADAM 和 RMSProp，它们更适合不同的模型和数据类型。

我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。


In [138]:
# 选择优化器
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分三个步骤进行：

-   调用optimizer.zero_grad()重置模型参数的梯度。默认情况下，梯度会累加；为了防止重复计数，我们在每次迭代中显式地将其归零。
-   使用loss.backward()反向传播预测损失。PyTorch 将损失相对于每个参数的梯度存储起来。
-   在获得梯度后，我们调用optimizer.step()根据反向传播中收集的梯度调整参数。


完整实现
===================

我们定义了train_loop来循环执行优化代码，以及test_loop来评估模型在测试数据上的性能。


In [139]:
# 前面定义好了部分组件
loss_fn=loss_fn
model=model
optimizer=optimizer

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 将模型设置为训练模式--这对批量归一化和丢弃层很重要
    # 虽然在这种情况下没有必要，但为最佳实践而添加
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # 计算预测值和损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward() # 计算梯度
        optimizer.step() # 更新梯度
        optimizer.zero_grad() # 梯度清空

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 将模型设置为评估模式--这对批量归一化和丢弃层很重要
    # 虽然在这种情况下没有必要，但为最佳实践而添加
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    
    # 使用 torch.no_grad() 评估模型可确保在测试模式下不计算梯度。
    # 还可减少不必要的梯度计算和内存使用，适用于 requires_grad=True 的张量系数
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化了损失函数和优化器，并将其传递给train_loop和test_loop。可以随意增加epoch数，以跟踪模型性能的提升。


In [140]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
loss: 2.379215  [   64/60000]
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
torch.Size([64, 12544])
torch.Size([64, 16, 28, 28])
